### Résumé des étapes dans ce notebook 01 :
- Chargement des données (images et légendes).
- Prétraitement des images (redimensionner, normaliser).
- Prétraitement des légendes (nettoyage, tokenisation, padding).
- Association des images et des légendes.
- Division en ensembles d’entraînement et de test.
- Sauvegarde des données.

In [34]:
# Importer les bibliothèques
import os
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import pickle


In [35]:
# Chemins des données
data_dir = "../datasets/flickr8k/"
images_dir = os.path.join(data_dir, "images/")
captions_file = os.path.join(data_dir, "captions.txt")

# Charger le fichier captions.txt
captions_df = pd.read_csv(captions_file, sep=",", header=None, names=["image", "caption"])

# Aperçu des données brutes
print("Aperçu des données brutes :")
print(captions_df.head())
print(f"Nombre total de légendes : {len(captions_df)}")


Aperçu des données brutes :
                       image  \
0                      image   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0                                            caption  
1  A child in a pink dress is climbing up a set o...  
2              A girl going into a wooden building .  
3   A little girl climbing into a wooden playhouse .  
4  A little girl climbing the stairs to her playh...  
Nombre total de légendes : 40456


In [36]:
# Vérifier que les fichiers d'images existent
def validate_image_paths(captions_df, images_dir):
    valid_images = []
    invalid_images = []

    for image in captions_df["image"].unique():
        image_path = os.path.join(images_dir, image)
        if os.path.exists(image_path):
            valid_images.append(image)
        else:
            invalid_images.append(image)

    print(f"Nombre d'images valides : {len(valid_images)}")
    print(f"Nombre d'images invalides : {len(invalid_images)}")
    return valid_images

valid_images = validate_image_paths(captions_df, images_dir)

# Garder uniquement les légendes associées aux images valides
captions_df = captions_df[captions_df["image"].isin(valid_images)].reset_index(drop=True)
print(f"Nombre de légendes après validation des images : {len(captions_df)}")


Nombre d'images valides : 8091
Nombre d'images invalides : 1
Nombre de légendes après validation des images : 40455


In [37]:
# Garder une seule légende par image (par exemple, la première)
unique_captions_df = captions_df.groupby("image").first().reset_index()

print("Aperçu des données uniques :")
print(unique_captions_df.head())
print(f"Nombre total d'images uniques : {len(unique_captions_df)}")


Aperçu des données uniques :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1001773457_577c3a7d70.jpg   
2  1002674143_1b742ab4b8.jpg   
3  1003163366_44323f5815.jpg   
4  1007129816_e794419615.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1         A black dog and a spotted dog are fighting  
2  A little girl covered in paint sits in front o...  
3  A man lays on a bench while his dog sits by him .  
4     A man in an orange hat starring at something .  
Nombre total d'images uniques : 8091


In [38]:
# Fonction pour nettoyer le texte
def clean_caption(caption):
    caption = caption.lower()  # Mettre en minuscule
    caption = re.sub(r"[^a-z0-9 ]", "", caption)  # Retirer les caractères spéciaux
    return caption

# Appliquer le nettoyage sur les légendes
unique_captions_df["cleaned_caption"] = unique_captions_df["caption"].apply(clean_caption)

print("Aperçu des légendes nettoyées :")
print(unique_captions_df[["image", "cleaned_caption"]].head())


Aperçu des légendes nettoyées :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1001773457_577c3a7d70.jpg   
2  1002674143_1b742ab4b8.jpg   
3  1003163366_44323f5815.jpg   
4  1007129816_e794419615.jpg   

                                     cleaned_caption  
0  a child in a pink dress is climbing up a set o...  
1         a black dog and a spotted dog are fighting  
2  a little girl covered in paint sits in front o...  
3   a man lays on a bench while his dog sits by him   
4      a man in an orange hat starring at something   


In [39]:
# Initialiser le tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<unk>")
tokenizer.fit_on_texts(unique_captions_df["cleaned_caption"])

# Convertir les légendes en séquences
unique_captions_df["caption_sequence"] = tokenizer.texts_to_sequences(unique_captions_df["cleaned_caption"])

# Padding des séquences
max_caption_length = 30
unique_captions_df["padded_sequence"] = pad_sequences(
    unique_captions_df["caption_sequence"], maxlen=max_caption_length, padding="post"
).tolist()

print("Aperçu des séquences de légendes :")
print(unique_captions_df[["image", "padded_sequence"]].head())


Aperçu des séquences de légendes :
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1001773457_577c3a7d70.jpg   
2  1002674143_1b742ab4b8.jpg   
3  1003163366_44323f5815.jpg   
4  1007129816_e794419615.jpg   

                                     padded_sequence  
0  [2, 38, 3, 2, 66, 144, 7, 124, 52, 2, 407, 9, ...  
1  [2, 12, 8, 5, 2, 754, 8, 17, 369, 0, 0, 0, 0, ...  
2  [2, 48, 15, 171, 3, 585, 101, 3, 41, 9, 2, 552...  
3  [2, 10, 622, 6, 2, 151, 27, 23, 8, 101, 46, 11...  
4  [2, 10, 3, 24, 82, 96, 1203, 19, 163, 0, 0, 0,...  


In [40]:
# Diviser les données en train/test
train_df, test_df = train_test_split(unique_captions_df, test_size=0.2, random_state=42)

# Créer les ensembles de données sous forme de tuples (chemin_image, légende_paddée)
train_data = [
    (os.path.join(images_dir, row["image"]), row["padded_sequence"]) for _, row in train_df.iterrows()
]
test_data = [
    (os.path.join(images_dir, row["image"]), row["padded_sequence"]) for _, row in test_df.iterrows()
]

print(f"Nombre d'images dans l'ensemble d'entraînement : {len(train_data)}")
print(f"Nombre d'images dans l'ensemble de test : {len(test_data)}")


Nombre d'images dans l'ensemble d'entraînement : 6472
Nombre d'images dans l'ensemble de test : 1619


In [41]:
# Sauvegarder les données d'entraînement et de test
with open("../datasets/flickr8k/train_data.pkl", "wb") as f:
    pickle.dump(train_data, f)

with open("../datasets/flickr8k/test_data.pkl", "wb") as f:
    pickle.dump(test_data, f)

print("Données prétraitées sauvegardées dans train_data.pkl et test_data.pkl.")


Données prétraitées sauvegardées dans train_data.pkl et test_data.pkl.
